# Counter-Strike ML - Round Swing on Major Games

This notebook is a machine learning personal project by [sadhubby](https://github.com/sadhubby) that analyses professional Counter Strike games during the StarLadder Budapest 2025 Major Playoffs and calculates round swing on actions done across the map each round.

As a high-level example, this is similar to Chess.com's move analyzer that calculates the possibility of winning for either black or white based on the move done by the player.

The intended use of this machine learning software is to be able to analyze one's own personal games in Counter-Strike, whether through Valve's matchmaking or FaceIt's third-party matchmaking. This is so that the user will have numerical and graphical evidence of how great an impact a certain action is to the round as a whole. 

First, importing [pandas](https://pandas.pydata.org/) library for data analysis and [awpy](https://github.com/pnxenopoulos/awpy) for the gathering the data from recorded professional Counter Strike games during the Major. The video files are taken from [HLTV](https://www.hltv.org/), a website for Counter Strike News and Coverage. They include the recorded videos, called match demos, for every game done during the Major. 

In [15]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

Before anything, it is first best to test this on a single map of a match. For the purpose of testing, Map 2 of the match between Vitality and FaZe Clan during the Grand Finals of StarLadder Budapest Major 2025.

Here, we first create a Demo instance of a match and parsing the instance.

In [ ]:
dem = Demo("vitality-vs-faze-m2-dust2.dem")
dem.parse()

Then we now gather the rounds of the map into `rounds_df`

In [ ]:
rounds_df = dem.rounds.to_pandas()


In [ ]:
print(f"Total Rounds Played: {len(rounds_df)}")

Next, we gather the amount of ticks that has happened in a round. Counter Strike 2 live gameplay happens at Sub-Tick, their new, and quite unwanted feature that came with the port to the Source 2 Engine. However, for the purposes of this project, the ticks will be recorded at 64 ticks.

In [ ]:
ticks = dem.ticks.to_pandas()
ticks['seconds_elapsed'] = ticks['tick'] / 64

In [ ]:
round_5_ticks = ticks[ticks['round_num'] == 5]

In [ ]:
round_5_ticks

In [ ]:
round_5_summary = round_5_ticks.groupby(['tick', 'seconds_elapsed', 'name'])['health'].sum().unstack()

In [ ]:
round_5_summary

Now, after having seen the rounds and ticks data frames and their respective columns, as well as see the parsing function work, we can now start creating the pipeline for the machine learning algorithm. 

Let us first settle the inputs and outputs. 
Our output, the outcome we want to see, is who will be the **winner** of a certain round given:
- the number of Counter-Terrorists (CT's) and Terrorist (T's) alive
- if the bomb is planted or not
- the time remaining in the round after certain events (such as eliminations, bomb plant)
- difference in equipment value (difference of the culmination of firepower and grenades from both CT's and T's)  